<a href="https://colab.research.google.com/github/PlayerDos/OneTwoThreeFour/blob/main/Load_By_Month.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime as dt
import numpy as np
import calendar

In [2]:
df = pd.read_csv('/content/needDSMYears.csv', index_col=0)
df.head()

,Code,Name,ParName,Calen,_31_03_22,_01_04_22,_02_04_22,_03_04_22,_04_04_22,_05_04_22,_06_04_22,_07_04_22,_08_04_22,_09_04_22,_10_04_22,_11_04_22,_12_04_22,_13_04_22,_14_04_22,_15_04_22,_16_04_22,_17_04_22,_18_04_22,_19_04_22,_20_04_22,_21_04_22,_22_04_22,_23_04_22,_24_04_22,_25_04_22,_26_04_22,_27_04_22,_28_04_22,_29_04_22,_30_04_22,_01_05_22,_02_05_22,_03_05_22,_06_05_22,_07_05_22,...,_22_11_22,_08_12_22,_09_12_22,_10_12_22,_11_12_22,_12_12_22,_13_12_22,_14_12_22,_15_12_22,_16_12_22,_17_12_22,_18_12_22,_19_12_22,_20_12_22,_21_12_22,_22_12_22,_23_12_22,_24_12_22,_25_12_22,_26_12_22,_27_12_22,_28_12_22,_29_12_22,_30_12_22,_31_12_22,_23_11_22,_24_11_22,_25_11_22,_26_11_22,_27_11_22,_28_11_22,_29_11_22,_30_11_22,_01_12_22,_02_12_22,_03_12_22,_04_12_22,_05_12_22,_06_12_22,_07_12_22
0,0,Аксай_5этап,Аксай_5этап,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Absm,Автобетоносмеситель,Автобетоносмеситель Количество,RT_24,16,11,8,5,4,3,3,3,6,6,9,9,8,9,9,6,1,1,1,4,4,4,4,4,5,1,1,2,2,5,8,8,8,8,2,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Abtnshk,Асфальтобетонщик,Асфальтобетонщик Количество,RT_20,50,35,25,20,20,19,15,10,9,14,18,9,8,18,14,10,10,10,9,15,15,4,0,4,9,13,5,5,4,4,5,9,9,9,10,18,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,AGud,Автогудронатор,Автогудронатор Количество,RT_24,10,3,2,2,1,2,1,2,0,1,1,0,1,1,2,1,1,1,2,1,1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,AP10,Асфальтоперегружатель,Асфальтоперегружатель Количество,RT_24,6,3,2,2,2,2,2,1,1,2,2,1,1,2,2,2,2,2,1,3,3,0,0,0,1,1,1,1,0,0,1,1,1,1,2,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
# форматирование имен столбцов из str в datetime ('_01_01_22' ---> 2022-01-01 00:00:00)
for index, item in enumerate(df.columns.values):
  df.columns.values[index] = pd.to_datetime(df.columns.values[index], format='_%d_%m_%y', errors='ignore')
  # форматирование имен столбцов из datetime в date (2022-01-01 00:00:00 ---> 2022-01-01)
  if type(df.columns.values[index]) is pd._libs.tslibs.timestamps.Timestamp:
    df.columns.values[index] = dt.date(df.columns.values[index])

In [4]:
# сортировка дней
df_Days = df.select_dtypes(exclude='object')
df_Info = df.select_dtypes(include='object')
df_Days.sort_index(axis=1, inplace=True)
df = df_Info.join(df_Days)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [5]:
# разбиение массива на 2 отдельных (количество техники и трудоёмкость)
df_Amount = pd.DataFrame(columns=df.columns)
df_WorkLoadPlan = pd.DataFrame(columns=df.columns)
for index, item in enumerate(df.values[:, 2]): # ??? change '2' to 'ParName' in enumerate(df.values[:, 2])
  if 'Количество' in item:
    df_Amount.loc[df_Amount.shape[0]] =  df.values[index]
  elif 'Трудоёмкость' in item:
    df_WorkLoadPlan.loc[df_WorkLoadPlan.shape[0]] = df.values[index]
  else:
    continue

РАСЧЕТ СРЕДНИХ ЗНАЧЕНИЙ ТЕХНИКИ ПО МЕСЯЦАМ

In [6]:
# форматирование фрейма для последующих операций
df_Amount.set_index('Name', inplace=True)
df_Amount = df_Amount.T

In [7]:
# получение списка значений "год-месяц" для последующего разделения фрейма по месяцам
year_month_list = []
for i in df_Amount.index[3:]:
  year_month = str(i)[:7]
  if (year_month not in year_month_list):
    year_month_list.append(year_month)

In [8]:
# создание пустого фрейма для средних значений техники по месяцам
df_Amount_Mean = pd.DataFrame(columns=df_Amount.columns)

for i in range(len(year_month_list)):
  df = pd.DataFrame(columns=df_Amount.columns)
  for index, item in enumerate(df_Amount.index):
    if year_month_list[i] in str(item):
      df = df.append(df_Amount.iloc[index])
  year = int(year_month_list[i][:4])
  month = int(year_month_list[i][5:7])
  last_day = calendar.monthrange(year, month)[1]

  # переменные с промежуточными датами
  d_01 = dt.strptime((year_month_list[i]+'-01'), '%Y-%m-%d').date()
  d_20 = dt.strptime((year_month_list[i]+'-20'), '%Y-%m-%d').date()
  d_10 = dt.strptime((year_month_list[i]+'-'+str(last_day-20)), '%Y-%m-%d').date()
  d_30 = dt.strptime((year_month_list[i]+'-'+str(last_day)), '%Y-%m-%d').date()

  # переменные с индексами элементов, соответствующих условиям
  idx_1 = df.index[(d_01 <= df.index) & (df.index <= d_20)].copy()
  idx_2 = df.index[(d_10 < df.index) & (df.index <= d_30)].copy()

  # фреймы с рассчитанными средними за первые и последние 20 дней
  df_1 = df.loc[idx_1].mean(axis=0)
  df_2 = df.loc[idx_2].mean(axis=0)

  # слияние с выбором максимального среднего в одну строку
  row_max_mean = pd.concat([df_1, df_2], axis=1).max(axis=1)
  row_max_mean.name = calendar.month_name[month]

  # добавление строки со рассчитанными средними значениями в итоговый фрейм
  df_Amount_Mean = df_Amount_Mean.append(row_max_mean)

# настройка отображения и форматирования значений фрейма
df_Amount_Mean = df_Amount_Mean.T
pd.options.display.float_format = '{:.2f}'.format

# форматирование значений месяцев
months = df_Amount_Mean.columns.values.tolist()
df_Amount_Mean.columns = pd.CategoricalIndex(df_Amount_Mean.columns, categories=months, ordered=True)

df_Amount_Mean.head()

,January,February,March,April,May,June,July,August,September,October,November,December
Name,,,,,,,,,,,,
Автобетоносмеситель,0.00,0.00,0.80,5.50,5.55,1.85,6.25,5.10,4.65,2.90,0.00,0.00
Асфальтобетонщик,0.86,0.00,2.50,15.15,10.75,4.05,15.60,21.80,14.50,11.50,1.95,0.00
Автогудронатор,0.00,0.00,0.50,1.30,0.95,0.40,0.95,1.80,1.20,0.85,0.15,0.00
Асфальтоперегружатель,0.00,0.00,0.30,1.90,1.40,0.45,1.20,2.10,1.45,1.35,0.20,0.00
"Автосамосвал, грузоподъемность от 22 до 27 т",29.07,32.00,24.25,55.30,70.20,55.85,96.85,80.30,76.60,47.30,34.30,2.95


РАСЧЕТ ТРУДОЁМКОСТИ ПО МЕСЯЦАМ

In [9]:
# форматирование фрейма для последующих операций
df_WorkLoadPlan.set_index('Name', inplace=True)
df_WorkLoadPlan = df_WorkLoadPlan.T

In [10]:
df_T = pd.DataFrame(columns=df_WorkLoadPlan.columns)

for i in range(len(year_month_list)):
  df = pd.DataFrame(columns=df_WorkLoadPlan.columns)
  for index, item in enumerate(df_WorkLoadPlan.index):
    if year_month_list[i] in str(item):
      df = df.append(df_WorkLoadPlan.iloc[index])
  year = int(year_month_list[i][:4])
  month = int(year_month_list[i][5:7])
  last_day = calendar.monthrange(year, month)[1]

  # переменные с датами первого и последнего дня месяца
  d_01 = dt.strptime((year_month_list[i]+'-01'), '%Y-%m-%d').date()
  d_30 = dt.strptime((year_month_list[i]+'-'+str(last_day)), '%Y-%m-%d').date()

  # переменные с индексами элементов (дат), входящих в месяц
  idx = df.index[(d_01 <= df.index) & (df.index <= d_30)].copy()

  # строка с рассчитанными суммами трудоёмкости за месяц
  row_sum_workloadplan = df.loc[idx].sum(axis=0)
  row_sum_workloadplan.name = calendar.month_name[month]

  # добавление строки со рассчитанными суммами трудоёмкости в итоговый фрейм
  df_T = df_T.append(row_sum_workloadplan)

# настройка отображения и форматирования значений фрейма
df_T = df_T.T
df_T = df_T.iloc[:].astype(int)

# форматирование названий месяцев
months = df_T.columns.values.tolist()
df_T.columns = pd.CategoricalIndex(df_T.columns, categories=months, ordered=True)

df_T.head()

,January,February,March,April,May,June,July,August,September,October,November,December
Name,,,,,,,,,,,,
Автобетоносмеситель,0,0,133,1286,1421,728,1476,1514,920,482,0,0
Асфальтобетонщик,81,0,605,3949,2896,1195,6264,7160,5545,3161,306,0
Автогудронатор,0,0,34,102,83,26,80,546,70,65,6,0
Асфальтоперегружатель,0,0,68,442,397,151,265,449,298,299,25,0
"Автосамосвал, грузоподъемность от 22 до 27 т",10480,14192,11945,25560,28929,25734,49368,40485,36689,19203,13621,1424


РАСЧЕТ ЗАГРУЗКИ

In [11]:
df_Load = df_T / (30 * 20 * df_Amount_Mean)
df_Load = df_Load.fillna(0)

In [12]:
# форматирование отображения числовых значений с плавающей точкой (1.000 ---> 100%)
for i in df_Load.columns:
  df_Load[i] = df_Load[i].map('{:.1%}'.format)

df_Load.head()

,January,February,March,April,May,June,July,August,September,October,November,December
Name,,,,,,,,,,,,
Автобетоносмеситель,0.0%,0.0%,27.7%,39.0%,42.7%,65.6%,39.4%,49.5%,33.0%,27.7%,0.0%,0.0%
Асфальтобетонщик,15.8%,0.0%,40.3%,43.4%,44.9%,49.2%,66.9%,54.7%,63.7%,45.8%,26.2%,0.0%
Автогудронатор,0.0%,0.0%,11.3%,13.1%,14.6%,10.8%,14.0%,50.6%,9.7%,12.7%,6.7%,0.0%
Асфальтоперегружатель,0.0%,0.0%,37.8%,38.8%,47.3%,55.9%,36.8%,35.6%,34.3%,36.9%,20.8%,0.0%
"Автосамосвал, грузоподъемность от 22 до 27 т",60.1%,73.9%,82.1%,77.0%,68.7%,76.8%,85.0%,84.0%,79.8%,67.7%,66.2%,80.5%


СЛИЯНИЕ РЕЗУЛЬТАТОВ

In [13]:
# добавление подназваний столбцов
df_Load.columns = pd.MultiIndex.from_product([["Загрузка"], df_Load.columns])
df_T.columns = pd.MultiIndex.from_product([["Трудоёмкость"], df_T.columns])
df_Amount_Mean.columns = pd.MultiIndex.from_product([["Количество"], df_Amount_Mean.columns])

df_Load = df_Load.swaplevel(axis=1)
df_T = df_T.swaplevel(axis=1)
df_Amount_Mean = df_Amount_Mean.swaplevel(axis=1)

In [14]:
# слияние фреймов загрузки и трудоёмкости
df_T_Load = pd.merge(df_Load, df_T, left_index=True, right_index=True)
df_T_Load.sort_index(axis=1, level=0, inplace=True)
df_T_Load.head()

January  ...     December
                                             Загрузка  ... Трудоёмкость
Name                                                   ...             
Автобетоносмеситель                              0.0%  ...            0
Асфальтобетонщик                                15.8%  ...            0
Автогудронатор                                   0.0%  ...            0
Асфальтоперегружатель                            0.0%  ...            0
Автосамосвал, грузоподъемность от 22 до 27 т    60.1%  ...         1424

[5 rows x 24 columns]

In [15]:
# слияние фреймов загрузки и количества
df_Amount_Load = pd.merge(df_Load, df_Amount_Mean, left_index=True, right_index=True)
df_Amount_Load.sort_index(axis=1, level=0, inplace=True)
df_Amount_Load.head()

January  ...   December
                                             Загрузка  ... Количество
Name                                                   ...           
Автобетоносмеситель                              0.0%  ...       0.00
Асфальтобетонщик                                15.8%  ...       0.00
Автогудронатор                                   0.0%  ...       0.00
Асфальтоперегружатель                            0.0%  ...       0.00
Автосамосвал, грузоподъемность от 22 до 27 т    60.1%  ...       2.95

[5 rows x 24 columns]

СОХРАНЕНИЕ ФАЙЛОВ

In [16]:
# сохранение результатов "Загрузка/Трудоёмкость" в csv-файл
df_T_Load.to_csv('T_Load_Data.csv')

In [17]:
# сохранение результатов "Загрузка/Трудоёмкость" в xls-файл
df_T_Load.to_excel('T_Load_Data.xlsx')

In [18]:
# сохранение результатов "Загрузка/Количество" в csv-файл
df_Amount_Load.to_csv('Amount_Load_Data.csv')

In [19]:
# сохранение результатов "Загрузка/Трудоёмкость" в xls-файл
df_Amount_Load.to_excel('Amount_Load_Data.xlsx')